In [ ]:
# 准备数据（XOR 的真值表）

import numpy as np

g_batch_size = 8

def Data():
    data = [
        [ [0, 0], [0] ],
        [ [0, 1], [1] ],
        [ [1, 0], [1] ],
        [ [1, 1], [0] ]
    ]

    return data


class DataLoader():
    def __init__(self, data):
        self.data = data
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, Index):
        np.random.shuffle(self.data) # 乱序
        #item = self.data[Index]
        item = self.data[0]
        return item[0], item[1]

def infinite_iter(data_loader, batch_size):
  it = iter(data_loader)
  while True:
    try:
      arr1 = []
      arr2 = []
      for i in range(batch_size):
        source , target = next(it)
        arr1.append( source )
        arr2.append( target )
      
      yield arr1, arr2
    except StopIteration:
      it = iter(data_loader)

train_loader = DataLoader(Data())

train_iter = infinite_iter(train_loader, g_batch_size)

X_0, Y = next(train_iter) # 随机得到三个样本，作为一个mini-batch 输入

print(X_0, Y)



In [ ]:
# 训练
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time

import os
from google.colab import drive


from torchvision import datasets, transforms
from torch import nn
from torch import optim

# Layer details for the neural network
input_size = 2
hidden_sizes = [2, 2]
output_size = 1

# Build a feed-forward network
model = nn.Sequential(nn.Linear(2, 5),
                      nn.ReLU(),
                      nn.Linear(5, 5),
                      nn.ReLU(),
                      nn.Linear(5, 2),
                      nn.LogSoftmax(dim=1)) # dim=1：对每一行的所有元素进行softmax运算，并使得每一行所有元素和为1


print(model)

optimizer = optim.SGD(model.parameters(), lr=0.003, momentum=0.9)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)

criterion = nn.NLLLoss()




time0 = time()
epochs = 1
for e in range(epochs):

    for i in range(10000):

        running_loss = 0


        X_0, Y = next(train_iter) # 随机得到三个样本，作为一个mini-batch 输入


        sources = torch.tensor(X_0, dtype=torch.float)
        labels = torch.tensor(Y)

        labels = labels.reshape(g_batch_size)
        labels = torch.tensor(labels, dtype=torch.long) 

        #import pdb; pdb.set_trace() # 调试， exit 退出

        #print(sources)
        #print(labels)

        
        # Training pass
        optimizer.zero_grad()
        
        output = model(  sources.cuda() if torch.cuda.is_available() else sources.cpu() )

        loss = criterion(output, labels.cuda() if torch.cuda.is_available() else labels.cpu())
        
        #This is where the model learns by backpropagating
        loss.backward()
        
        #And optimizes its weights here
        optimizer.step()
        
        running_loss += loss.item()


    print("Epoch {} - Training loss: {}".format(e, running_loss/3))
    print("\nTraining Time (in minutes) =",(time()-time0)/60)


In [ ]:
# 验证模型的正确率

correct_count, all_count = 0, 0
X_0, Y = next(train_iter)

sources = torch.tensor(X_0, dtype=torch.float)

labels = torch.tensor(Y)
labels = labels.reshape(g_batch_size)
labels = torch.tensor(labels, dtype=torch.long) 


with torch.no_grad():
  for i in range(len(sources)):
    src = sources[i].reshape(1, 2) # images[i].reshape(1, 784)
    logps = model( src.cuda() if torch.cuda.is_available() else src.cpu() )

    # Output of the network are log-probabilities, need to take exponential for probabilities
    ps = torch.exp(logps)

    #import pdb; pdb.set_trace() # 调试， exit 退出

    probab = list(ps.cpu().numpy()[0])
    pred_label = probab.index(max(probab))


    #img_to_show = images[i].reshape(28,28)
    #img_to_show = img_to_show * 255

    #plt.imshow(img_to_show, cmap='gray_r'); 

    print(pred_label)  # 预测的数字

    true_label = labels.numpy()[i]

    print(true_label)  # 真实的数字

    if(true_label == pred_label):
      correct_count += 1
    all_count += 1


print("Number Of Images Tested =", all_count)
print("\nModel Accuracy =", (correct_count/all_count))  # 预测的正确率

